In [10]:
!pip install findspark

In [11]:
import os
import sys
os.environ['SPARK_HOME']=r'/home/ritika/Downloads/spark-3.0.0-bin-hadoop2.7'

In [12]:
from __future__ import print_function
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.mllib.feature import StandardScaler,PCA

In [13]:
if __name__ == "__main__":
    spark = SparkSession .builder\
        .appName("PCA")\
        .getOrCreate()

In [14]:
df = spark.read.csv("glass.data",header=True)

In [15]:
df.show()

+-----+-------+-----+----+----+-----+----+----+----+----+-----+
|index|     RI|   Na|  Mg|  Al|   Si|   K|  Ca|  Ba|  Fe|Class|
+-----+-------+-----+----+----+-----+----+----+----+----+-----+
|    1|1.52101|13.64|4.49|1.10|71.78|0.06|8.75|0.00|0.00|    1|
|    2|1.51761|13.89|3.60|1.36|72.73|0.48|7.83|0.00|0.00|    1|
|    3|1.51618|13.53|3.55|1.54|72.99|0.39|7.78|0.00|0.00|    1|
|    4|1.51766|13.21|3.69|1.29|72.61|0.57|8.22|0.00|0.00|    1|
|    5|1.51742|13.27|3.62|1.24|73.08|0.55|8.07|0.00|0.00|    1|
|    6|1.51596|12.79|3.61|1.62|72.97|0.64|8.07|0.00|0.26|    1|
|    7|1.51743|13.30|3.60|1.14|73.09|0.58|8.17|0.00|0.00|    1|
|    8|1.51756|13.15|3.61|1.05|73.24|0.57|8.24|0.00|0.00|    1|
|    9|1.51918|14.04|3.58|1.37|72.08|0.56|8.30|0.00|0.00|    1|
|   10|1.51755|13.00|3.60|1.36|72.99|0.57|8.40|0.00|0.11|    1|
|   11|1.51571|12.72|3.46|1.56|73.20|0.67|8.09|0.00|0.24|    1|
|   12|1.51763|12.80|3.66|1.27|73.01|0.60|8.56|0.00|0.00|    1|
|   13|1.51589|12.88|3.43|1.40|73.28|0.6

In [16]:
# we will dos tandard scaling before applying PCA

In [17]:
#we do not need to scale our label coulmn, so let's seperate it

label = df.select('Class')

In [18]:
features = df.drop('Class')

In [19]:
col_names = features.columns
features_rdd = features.rdd.map(lambda row: row[0:])

In [20]:
features_rdd.collect()

[('1',
  '1.52101',
  '13.64',
  '4.49',
  '1.10',
  '71.78',
  '0.06',
  '8.75',
  '0.00',
  '0.00'),
 ('2',
  '1.51761',
  '13.89',
  '3.60',
  '1.36',
  '72.73',
  '0.48',
  '7.83',
  '0.00',
  '0.00'),
 ('3',
  '1.51618',
  '13.53',
  '3.55',
  '1.54',
  '72.99',
  '0.39',
  '7.78',
  '0.00',
  '0.00'),
 ('4',
  '1.51766',
  '13.21',
  '3.69',
  '1.29',
  '72.61',
  '0.57',
  '8.22',
  '0.00',
  '0.00'),
 ('5',
  '1.51742',
  '13.27',
  '3.62',
  '1.24',
  '73.08',
  '0.55',
  '8.07',
  '0.00',
  '0.00'),
 ('6',
  '1.51596',
  '12.79',
  '3.61',
  '1.62',
  '72.97',
  '0.64',
  '8.07',
  '0.00',
  '0.26'),
 ('7',
  '1.51743',
  '13.30',
  '3.60',
  '1.14',
  '73.09',
  '0.58',
  '8.17',
  '0.00',
  '0.00'),
 ('8',
  '1.51756',
  '13.15',
  '3.61',
  '1.05',
  '73.24',
  '0.57',
  '8.24',
  '0.00',
  '0.00'),
 ('9',
  '1.51918',
  '14.04',
  '3.58',
  '1.37',
  '72.08',
  '0.56',
  '8.30',
  '0.00',
  '0.00'),
 ('10',
  '1.51755',
  '13.00',
  '3.60',
  '1.36',
  '72.99',
  '0.57',


In [21]:
scaler1 = StandardScaler().fit(features_rdd)

In [22]:
scaled_features=scaler1.transform(features_rdd)

In [23]:
for data in scaled_features.collect():
    print(data)

[0.01614970175477225,500.848945006586,16.703331628353496,3.112850513096934,2.203218260299005,92.67366821321163,0.09199747038936874,6.148317857629576,0.0,0.0]
[0.0322994035095445,499.72936892686107,17.009477735911293,2.4958266920153593,2.723978940006043,93.90019349605575,0.73597976311495,5.501866151455952,0.0,0.0]
[0.048449105264316755,499.2584883992121,16.568627341028062,2.4611624324040347,3.084505564418607,94.2358740997815,0.5979835575308969,5.466732906555212,0.0,0.0]
[0.064598807019089,499.7458332809747,16.17676032335408,2.558222359315743,2.583774141623379,93.74526398664386,0.8739759686990031,5.775905461681728,0.0,0.0]
[0.08074850877386125,499.66680438122944,16.25023538916795,2.509692395859889,2.483627857064333,94.35207123184043,0.8433101452358803,5.670505726979506,0.0,0.0]
[0.09689821052863351,499.18604524111225,15.662434862656978,2.502759543937624,3.2447396197130804,94.21005251487952,0.9813063508199333,5.670505726979506,0.0,2.668344285192604]
[0.11304791228340576,499.6700972520522,

In [24]:
pca = PCA(k=3)
model = pca.fit(scaled_features)

In [25]:
result = model.transform(scaled_features)

In [26]:
#store dense vector in a dataframe

In [27]:
df =result.map(lambda x: (x, )).toDF(["PCA_Features"])

In [28]:
df.show(truncate=False)

+------------------------------------------------------------+
|PCA_Features                                                |
+------------------------------------------------------------+
|[121.27139072423623,-255.77115647964797,-4.196845117326806] |
|[120.05550001159102,-254.54587403025658,-4.381436696803565] |
|[119.84798723248252,-254.2034447726247,-4.381251734901793]  |
|[120.46109000494367,-254.67949604517653,-3.945938300455288] |
|[120.35044014976697,-254.4653916330586,-4.314355447417802]  |
|[120.45089631994485,-254.37457759401596,-2.9689232295465304]|
|[120.42061242849479,-254.51736817101582,-4.3703267360499085]|
|[120.56003194921826,-254.53042060939444,-4.455718057710778] |
|[120.2159683626838,-255.23024548793205,-3.932357102246052]  |
|[120.5383776751363,-254.73937062375217,-3.7123364202676385] |
|[120.36864290561546,-254.29818102077232,-3.14870060655128]  |
|[120.55406978821279,-254.69820223347122,-3.976890708501814] |
|[120.42918759536727,-254.3202896257257,-3.352150806775

In [29]:
sc.stop()